## Dependencies

In [2]:
import pandas as pd
import geopandas as gpd
import pickle
import matplotlib.pyplot as plt
from shapely.ops import linemerge

## Parameters

In [8]:
HOME = 'C:\\Users\\BPARK17\\OneDrive - azureford\\JupyterNotebooks\\Elevate_dashbord\\'

SPATIAL_HOME = 'C:\\Users\\BPARK17\\OneDrive - azureford\\JupyterNotebooks\\Census_CitySolutions\\data\\census_areas_2020\\'

NETWORK_HOME = 'C:\\Users\\BPARK17\\OneDrive - azureford\\JupyterNotebooks\\ESRI_Network\\'
NETWORK_DATA = f'{NETWORK_HOME}data\\'
GRID = f'{NETWORK_DATA}Grid\\Grid_network\\'
PROJECT = 'Move'
USECASE = f'{NETWORK_DATA}UseCase\\{PROJECT}\\'

CRS = 'EPSG:4269'

VOT_HOUR = 8

## Functions

In [4]:
import datetime

def summary_table_preprocessing(table_path, data_dict_path):
  ## Input
  # read data
  data_dict_df = pd.read_csv(data_dict_path)
  sum_raw_df = pd.read_csv(table_path, sep='delimiter', engine='python')

  ## Data dictionary
  # categorize columns by type
  cols_int = [c for c in data_dict_df.loc[data_dict_df['data_type'].isin(['int', 'bigint'])]['col_name'].to_list() if c in sum_raw_df.columns.values[0].split(',')]
  cols_float = [c for c in data_dict_df.loc[data_dict_df['data_type']=='double']['col_name'].to_list() if c in sum_raw_df.columns.values[0].split(',')]
  cols_datetime = [c for c in data_dict_df.loc[data_dict_df['data_type'].isin(['timestamp'])]['col_name'].to_list() if c in sum_raw_df.columns.values[0].split(',')]
  cols_date = [c for c in data_dict_df.loc[data_dict_df['data_type'].isin(['date'])]['col_name'].to_list() if c in sum_raw_df.columns.values[0].split(',')]

  ## Summary table
  sum_list = []
  for rec in sum_raw_df.iterrows():
    rec_list = rec[1].values[0].split(',')
    if len(rec_list) > len(data_dict_df):    
      new_rec_list = []
      tmp_v = ''
      append = False
      for v in rec_list:
        if v[0] == '[':
          append = True
        if v[-1] == ']':
          append = False

        tmp_v += v
        if append == False:
          if len(tmp_v) == 0:
            new_rec_list.append(v)
          else:
            new_rec_list.append(tmp_v)
            tmp_v = ''
      sum_list.append(new_rec_list)
    else:
      sum_list.append(rec_list)
  sum_df = pd.DataFrame(sum_list)
  sum_df.columns = sum_raw_df.columns.to_list()[0].split(',')


  ## Convert data type
  for c in cols_int:
    sum_df[c] = sum_df[c].replace('NULL', -999).astype(int)

  for c in cols_float:
    sum_df[c] = sum_df[c].replace('NULL', -999).astype(float)

  for c in cols_datetime:
    rec_list = []
    for v in sum_df[c]:
      try:
        rec_list.append(datetime.datetime.strptime(v.split('.')[0], '%Y-%m-%d %H:%M:%S'))
      except:
        rec_list.append(datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S'))
    sum_df[c] = rec_list

  for c in cols_date:
    rec_list = []
    for v in sum_df[c]:
      try:
        rec_list.append(datetime.datetime.strptime(v, '%Y-%m-%d').date())
      except:
        rec_list.append(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
    sum_df[c] = rec_list

  sum_df = sum_df.sort_values(by=['cvdcqa_vin_d_3', 'cvdcqa_trip_no_r_3'])
  return sum_df

In [5]:
from shapely.geometry import MultiLineString
from shapely.ops import linemerge, unary_union

def convert_multi_to_single_line(geom):
    if isinstance(geom, MultiLineString):
        merged_line = linemerge(geom)
        result = unary_union(merged_line)
        return result[0]
    else:
        return geom

In [11]:
def get_trip_by_vot(gdf):
  date_list = []
  trip_id = 0
  trip_vot_dict = {}
  for rec in gdf.itertuples():
    
    if rec.cvdcqa_partition_date not in date_list:
      trip_id = 0
      date_list.append(rec.cvdcqa_partition_date)
      
    if f'{rec.cvdcqa_vin_d_3}_{rec.cvdcqa_partition_date}_{trip_id}' not in trip_vot_dict:
      trip_vot_dict[f'{rec.cvdcqa_vin_d_3}_{rec.cvdcqa_partition_date}_{trip_id}'] = []

    trip_vot_dict[f'{rec.cvdcqa_vin_d_3}_{rec.cvdcqa_partition_date}_{trip_id}'].append(rec)

    if rec.cvdcqa_time_spnt_at_dest_in_secs_r_3 > VOT_HOUR * 3600:
      trip_id +=1
  out = {key:gpd.GeoDataFrame(pd.DataFrame(val), geometry='geometry', crs=CRS) for key, val in trip_vot_dict.items()}
  return out

In [9]:
def get_multiple_routes(gdf):
  miles = 0
  geometry = []
  for ind in gdf.index:
    if ind < gdf.index.max():
      try:
        orig_lon = gdf.iloc[ind]['geometry'].coords[0][0]
        orig_lat = gdf.iloc[ind]['geometry'].coords[0][1]
        dest_lon = gdf.iloc[ind+1]['geometry'].coords[0][0]
        dest_lat = gdf.iloc[ind+1]['geometry'].coords[0][1]
        tot_route = sp.find_shortest_path(orig_lon, orig_lat, dest_lon, dest_lat, orig_knn=2, dest_knn=2)
        
        miles+=tot_route['miles']
        geometry.append(convert_multi_to_single_line(tot_route['geometry']))
      except:
        continue
  
  new_geometry = linemerge(geometry)
  return miles, new_geometry

In [55]:
from shapely.geometry import Point, LineString
import numpy as np
def points_to_line(point_gdf):
  stack_lonlat = point_gdf.agg({'cvdcqa_trip_orig_long_r_3': np.stack, 'cvdcqa_trip_orig_lat_r_3':  np.stack})
  lineStringObj = LineString(list(zip(*stack_lonlat)))

  rec_dict = {'cvdcqa_vin_d_3':[point_gdf.iloc[0]['cvdcqa_vin_d_3']],
              'cvdcqa_trip_strt_odo_read_r_3':[point_gdf.iloc[0]['cvdcqa_trip_strt_odo_read_r_3']],
              'cvdcqa_trip_end_odo_read_r_3':[point_gdf.iloc[-1]['cvdcqa_trip_end_odo_read_r_3']],
              'cvdcqa_trip_strt_time_s_3':[point_gdf.iloc[0]['cvdcqa_trip_strt_time_s_3']],
              'cvdcqa_trip_end_time_s_3':[point_gdf.iloc[-1]['cvdcqa_trip_end_time_s_3']],
              'cvdcqa_trip_durn_in_secs_r_3':[point_gdf['cvdcqa_trip_durn_in_secs_r_3'].sum()],
              'cvdcqa_time_spnt_at_dest_in_secs_r_3':[point_gdf.iloc[:-1]['cvdcqa_time_spnt_at_dest_in_secs_r_3'].sum()],
              'cvdcqa_odo_chng_km_r_3':[point_gdf['cvdcqa_odo_chng_km_r_3'].sum()],
              'cvdcqa_trip_orig_lat_r_3':[point_gdf.iloc[0]['cvdcqa_trip_orig_lat_r_3']],
              'cvdcqa_trip_orig_long_r_3':[point_gdf.iloc[0]['cvdcqa_trip_orig_long_r_3']],
              'cvdcqa_trip_dest_lat_r_3':[point_gdf.iloc[-1]['cvdcqa_trip_dest_lat_r_3']],
              'cvdcqa_trip_dest_long_r_3':[point_gdf.iloc[-1]['cvdcqa_trip_dest_long_r_3']],
              'geometry':lineStringObj
              }
  return gpd.GeoDataFrame(rec_dict)

In [66]:
vin_trip_straight_gdf = pd.DataFrame()
scav_df = pd.read_csv(f'{HOME}src\\data\\scav\\ncvdvqaa_trip_sum_4g_na_usa_msi_vw_2023_04_12.csv')
scav_gdf = gpd.GeoDataFrame(scav_df, geometry=gpd.points_from_xy(scav_df['cvdcqa_trip_orig_long_r_3'], scav_df['cvdcqa_trip_orig_lat_r_3']), crs=CRS)
for vin, vin_trip_gdf in scav_gdf.groupby('cvdcqa_vin_d_3'):
  vin_trips_dict = get_trip_by_vot(vin_trip_gdf.sort_values(by='cvdcqa_trip_strt_time_s_3'))
  for key, val_gdf in vin_trips_dict.items():
    val_gdf = val_gdf.loc[(val_gdf['cvdcqa_trip_orig_lat_r_3']!=-999)  & (val_gdf['cvdcqa_trip_orig_long_r_3']!=-999) & (~val_gdf['cvdcqa_trip_orig_long_r_3'].isna())]
    if len(val_gdf) > 2:
      vin_trip_straight_gdf = pd.concat([vin_trip_straight_gdf, points_to_line(val_gdf)])


In [69]:
vin_trip_straight_gdf.to_file(f'{HOME}src\\data\\scav\\test.shp')

C:\Users\BPARK17\AppData\Local\Temp/ipykernel_12276/2291085618.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  vin_trip_straight_gdf.to_file(f'{HOME}src\\data\\scav\\test.shp')
